In [1]:
import pandas as pd

In [14]:
def agregar_weas(df, columna):
    df[columna] = df[columna].apply(lambda x: f'| {x} |')
    return df

In [15]:
datos_dern = pd.read_csv('Src/Corpus/Corpus_Dern.csv')
datos_dern

,Mensaje
0,._.xD
1,Jugamos al rato?
2,Y como descargo más stickers xd
3,Si ese ya lo tengo
4,Pero no hay más?
...,...
12233,a
12234,es que hoy me funan
12235,JAJAJAJAJA
12236,(Rie para no llorar)


In [16]:
datos_esme = pd.read_csv('Src/Corpus/Corpus_Esme.csv')
datos_esme

,Mensaje
0,Hoy me toco tremendo maraton
1,De hecho jajajaj
2,Nooo
3,Niyo
4,Y trngo un bn de tarea
...,...
1561,Cualeeees?
1562,Porfiiii
1563,Yendo
1564,Sacaaaaa


In [17]:
datos_dern=agregar_weas(datos_dern, 'Mensaje')
datos_esme=agregar_weas(datos_esme, 'Mensaje')

In [18]:
datos_esme

,Mensaje
0,| Hoy me toco tremendo maraton |
1,| De hecho jajajaj |
2,| Nooo |
3,| Niyo |
4,| Y trngo un bn de tarea |
...,...
1561,| Cualeeees? |
1562,| Porfiiii |
1563,| Yendo |
1564,| Sacaaaaa |


In [19]:
datos_dern.to_csv('Src/Corpus/Corpus_Dern.csv', index=False)

In [20]:
datos_esme.to_csv('Src/Corpus/Corpus_Esme.csv', index=False)